# Flame animation
This notebook explains the process of creating the fire animation for the LEDs.

The principle behind the animation is simple and relies on three components:
1. A Perlin Noise generator
2. A vertical gradient mask
3. A colormap to convert floats $\in[0,1]$ to RGB colours.

By far, the most complex component is the Perlin noise generator. Perlin noise is a type or procedural noise that can be used to generate psudo-fractal noise.

Our implementation is based on the [`perlin-numpy`](https://github.com/pvigier/perlin-numpy) module. Its author also has a great [blog post](https://pvigier.github.io/2018/06/13/perlin-noise-numpy.html) exmplaining the generation algorithm. 

This module does not use `perlin-numpy` directly because the initial approach demanded additional functionality not present in `perlin-numpy`. In particular, it needed to dynamically create scrollable noise.


The cell below shows Perlin noise in different grid sizes, including the 8x8 grid used for the animation.


In [ ]:
from time import sleep
from matplotlib import pyplot as plt
import numpy as np
from fireplace.lights.noise import PerlinNoise

shapes = [200,20,8]

fig, ax = plt.subplots(nrows=1,ncols=len(shapes),figsize = (10,5), dpi = 100)

for i,shape in enumerate(shapes):
    perlin = PerlinNoise(shape=(shape,shape),seed=75)
    noise = perlin.render(octaves=7)
    ax[i].imshow(noise.T)
    ax[i].set_title(f"Perlin noise of shape {shape}x{shape}")
    ax[i].set_xticks([])
    ax[i].set_yticks([])

This is then combined with a gradient filter to simulate the cooling down of the flame as it raises

In [ ]:
from fireplace.lights.noise import quadratic_mask

shape = (20,20)


vertical_filter = quadratic_mask((20,20), 0.6, 1.2)
perlin = PerlinNoise(shape=shape,seed=75)
noise = perlin.render(octaves=7)
image = (noise * vertical_filter)


fig, ax = plt.subplots(nrows=1,ncols=len(shapes),figsize = (10,5), dpi = 100)


ax[0].imshow(noise.T,vmax=1,vmin=0)
ax[0].set_title(f"Perlin noise")
ax[0].set_xticks([])
ax[0].set_yticks([])

ax[1].imshow(vertical_filter.T,vmax=1,vmin=0)
ax[1].set_title(f"Vertical filter")
ax[1].set_xticks([])
ax[1].set_yticks([])

ax[2].imshow(image.T,vmax=1,vmin=0)
ax[2].set_title(f"noise x filter")
ax[2].set_xticks([])
ax[2].set_yticks([])



The third and last component is the colour, obtained by using a colourmap to give a realistic colour to the flame

In [ ]:
from fireplace.lights.utils import ColorMap, hex_to_rgb

HEX_PALETTE = [
    "1f0900",
    "54370b",
    "754b03",
    "8e5318",
    "ad5c00",
    "d97b09",
    "fa9a2c",
    "fcb308",
]
# https://coolors.co/1f0900-54370b-754b03-8e5318-ad5c00-d97b09-fa9a2c-fcb308
rgb_palette = [hex_to_rgb(color) for color in HEX_PALETTE]

colormap = ColorMap(palette=rgb_palette)

plt.imshow(colormap(image.T))
plt.xticks([])
plt.yticks([])


The only thing that remains is to simulate the vertical movement of the flame. This turns out to be the most complicated step of the process

## Dynamic genneration [too slow]

The original ideas was to dynamically generate new rows of noise to generate the vertical movement. This was achieved using the `pixel_offset` parameter, which displaces the grid by a set amount.

In [ ]:
from time import sleep
from IPython.display import clear_output

size = (8, 8)

perlin = PerlinNoise(shape=size, repetition_period=200)
filter = quadratic_mask(size, 0.6, 1.2)

for shift in range(100):
    noise = perlin.render(octaves=4, pixel_offset=(0, shift), relative_factor=2)
    image = (noise * filter).T
    image = colormap(image)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    plt.show()  
    sleep(0.05)  
    clear_output(wait=True)  # clear the output for the next loop


Unfortunately this turned out to be too slow to get a decent frame rate in the Raspberry Pi.
The code runs really quickly on a laptop, but it is slower on the pi. Moreover, the action of sending the data to the LEDs already takes a relatively long time every cycle, so there is not much margin for lenghtly calculations.  

In [ ]:
%%time

for shift in range(10_000):
    noise = perlin.render(octaves=4,pixel_offset=(0,shift))

# CPU times: user 6.56 s, sys: 5.48 ms, total: 6.57 s
# Wall time: 6.59 s

# Pre-computed noise approach

The solution was to generate the noise in the installation process, and load it at runtime. Below we just show a portion of the strip.

In [ ]:
from fireplace.lights.noise import load_noise, noise_files_dir, generate_noise_files


# run the line below if the noise files have not been generated yet
# generate_noise_files(noise_files_dir=noise_files_dir,n_files=6,length=30 * 60 * 1)
noise = load_noise(noise_files_dir)
noise_back = load_noise(noise_files_dir)

# now the noise array contains a long strip of noise

plt.figure(figsize = (5,20))
plt.imshow(noise[:50],interpolation="nearest")
plt.xticks([])
plt.yticks([])


At runtime, we only show 8 rows at a time:

In [ ]:
#%%time

window = 8
max_step = noise.shape[0]-window
step = 0
for i in range(200):
    if step < max_step:
        screen = noise[step:(step+window)]
    else:
        step = 0
        noise = noise_back
        max_step = noise.shape[0]-window
        noise_back = load_noise(noise_files_dir)
        screen = noise[step:(step+window)]
    step+=1
    image = screen*filter.T
    image = colormap(image)
    # show
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    plt.show()  # show the plot
    sleep(0.05)
    clear_output(wait=True)  # clear the output for the next loop



We can see that this is much faster, by an order of magnitude, than the dynamic generation

In [ ]:
%%time

window = 8
max_step = noise.shape[0]-window
step = 0
for i in range(10_000):
    if step < max_step:
        screen = noise[step:(step+window)]
    else:
        step = 0
        noise = noise_back
        max_step = noise.shape[0]-window
        noise_back = load_noise(noise_files_dir)
        screen = noise[step:(step+window)]
    step+=1
    image = screen*filter.T
    image = colormap(image)


# CPU times: user 304 ms, sys: 3.47 ms, total: 308 ms
# Wall time: 312 ms
